## Import Libraries

In [45]:
import numpy as np
import pandas as pd
import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report

np.random.seed(13)
print(sklearn.__version__)
print(np.__version__)
print(pd.__version__)

1.6.1
2.2.0
2.2.3


## Import data

In [2]:
df = pd.read_csv("../data/JFP Credential Data_All_Journals_Final_1.2025.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         5630 non-null   object
 1   Abstract      4046 non-null   object
 2   Authors       5466 non-null   object
 3   documentType  5630 non-null   object
 4   pub_month     5337 non-null   object
 5   pub_year      5630 non-null   int64 
 6   pubtitle      5630 non-null   object
 7   subjectTerms  4067 non-null   object
 8   Database      5630 non-null   object
dtypes: int64(1), object(8)
memory usage: 396.0+ KB


## Extract Credentials

In [4]:
df["Credentials"] = (df
                     .Authors
                     .apply(lambda x: re.findall(r"(CFP|PhD|CFA|JD|CPA|CLU|ChFC|LLM|AIF|CRC)", str(x)))
                     .apply(lambda x: np.nan if len(x) == 0 else x)
                     )

## Group Credentials into target groups

In [5]:
def target_grouping(x):
    if type(x) != list:
        return np.nan
    elif "PhD" in set(x):
        return "Academic"
    else:
        return "Practitioner"

df["target_grouping"] = df.Credentials.apply(target_grouping)

## Drop nulls

In [6]:
df = df.dropna(subset=["target_grouping", "Abstract"]).reset_index(drop=True)

In [7]:
df.head()

,Title,Abstract,Authors,documentType,pub_month,pub_year,pubtitle,subjectTerms,Database,Credentials,target_grouping
0,How Do Commodities Fit into Client Portfolios?,The standard stock--bond portfolio mix remains...,"Fink, Jason D, PhD;Fink, Kristin E, PhD",Commentary,Oct,2022,Journal of Financial Planning,"Financial planning , Investment , Bonds , Sto...",ProQuest One Academic,"[PhD, PhD]",Academic
1,The Role of Financial Planners on African Amer...,"* Addressing the issue of ""wealth gap,"" partic...","Chen, Leon, PhD, FRM;Duffy, Sophia, JD, CPA;Hi...",Commentary,Nov,2022,Journal of Financial Planning,"Credit , Small business , Wealth distribution...",ProQuest One Academic,"[PhD, JD, CPA, PhD, CFP]",Academic
2,"How to Effectively Manage Six ""Oh Bleep"" Clien...",Managing difficult client situations is part o...,"Heye, Chris, PhD",Journal Article,May,2022,Journal of Financial Planning,"Interest rates , Financial management , Finan...",ProQuest One Academic,[PhD],Academic
3,Inflation Uncertainty Calls for Portfolio Prot...,"Inflation currently sits at 7.9 percent, havin...","Marotta, Ryann, CFP®, CFA",Journal Article,May,2022,Journal of Financial Planning,"Investors , Investments , Pandemics , Monetar...",ProQuest One Academic,"[CFP, CFA]",Practitioner
4,Financial Planners' Gray Divorce Checklist,Divorce today is less common for adults younge...,"Stephenson, Angie M, CFP®, CPA/PFS",Journal Article,May,2022,Journal of Financial Planning,"Divorce , Marital separation , Marriage , Tax...",ProQuest One Academic,"[CFP, CPA]",Practitioner


In [8]:
X = df.Abstract
y = df.target_grouping

In [9]:
X.shape, y.shape

((2608,), (2608,))

In [10]:
y.value_counts()

target_grouping
Practitioner    1620
Academic         988
Name: count, dtype: int64

In [11]:
y.value_counts().to_csv("../results/binary_target_distribution.csv")

## Encode Target

In [12]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [13]:
encoder.classes_

array(['Academic', 'Practitioner'], dtype=object)

## Create Pipeline

In [34]:
models = [LogisticRegression(class_weight="balanced"), 
          DecisionTreeClassifier(class_weight="balanced"), 
          RandomForestClassifier(class_weight="balanced"),
          HistGradientBoostingClassifier(class_weight="balanced")]

# Define the custom transformer
class DenseTransformer:
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # make the array dense
        X = X.toarray()
        return X

pipelines = [Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("densify", DenseTransformer()),
    ("classifier", model)
    ]) for model in models]

In [36]:
pipelines

[Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x00000267AF92BEC0>),
                 ('classifier', LogisticRegression(class_weight='balanced'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x00000267AF89BE00>),
                 ('classifier',
                  DecisionTreeClassifier(class_weight='balanced'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x00000267AF898FE0>),
                 ('classifier',
                  RandomForestClassifier(class_weight='balanced'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x00000267AF89BC20>),
                 ('classifier',
                  HistGradientBoostingClassifier(cla

## Cross Validation

In [37]:
results = [cross_validate(pipeline, X, y, scoring=["precision", "recall", "accuracy", "f1"], cv=5, n_jobs=-1) for pipeline in pipelines]

In [38]:
result_df = pd.DataFrame({
    "model": models,
    "avg_precision": [result["test_precision"].mean() for result in results],
    "avg_recall": [result["test_recall"].mean() for result in results],
    "avg_f1": [result["test_f1"].mean() for result in results],
    "avg_accuracy": [result["test_accuracy"].mean() for result in results]
})

In [39]:
result_df

,model,avg_precision,avg_recall,avg_f1,avg_accuracy
0,LogisticRegression(class_weight='balanced'),0.818879,0.800000,0.809124,0.766089
1,DecisionTreeClassifier(class_weight='balanced'),0.725217,0.718519,0.721650,0.656045
2,RandomForestClassifier(class_weight='balanced'),0.758841,0.917901,0.830509,0.767620
3,HistGradientBoostingClassifier(class_weight='b...,0.804640,0.829012,0.816329,0.768393


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [44]:
print(f"Train set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

Train set shape: (1956,)
Test set shape: (652,)


In [49]:
lr_pipeline = pipelines[0]

lr_pipeline.fit(X_train, y_train)

y_pred = lr_pipeline.predict(X_test)

lr_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Logistic Regression Classification Report")
lr_report


Logistic Regression Classification Report


,Academic,Practitioner,accuracy,macro avg,weighted avg
precision,0.736264,0.878628,0.819018,0.807446,0.824696
recall,0.813765,0.822222,0.819018,0.817994,0.819018
f1-score,0.773077,0.849490,0.819018,0.811283,0.820542
support,247.000000,405.000000,0.819018,652.000000,652.000000


In [50]:
dt_pipeline = pipelines[1]

dt_pipeline.fit(X_train, y_train)

y_pred = dt_pipeline.predict(X_test)

dt_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Decision Tree Classification Report")
dt_report

Decision Tree Classification Report


,Academic,Practitioner,accuracy,macro avg,weighted avg
precision,0.552632,0.740933,0.66411,0.646782,0.669598
recall,0.595142,0.706173,0.66411,0.650657,0.664110
f1-score,0.573099,0.723135,0.66411,0.648117,0.666297
support,247.000000,405.000000,0.66411,652.000000,652.000000


In [51]:
rf_pipeline = pipelines[2]

rf_pipeline.fit(X_train, y_train)

y_pred = rf_pipeline.predict(X_test)

rf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Random Forest Classification Report")
rf_report

Random Forest Classification Report


,Academic,Practitioner,accuracy,macro avg,weighted avg
precision,0.827381,0.776860,0.789877,0.802120,0.795999
recall,0.562753,0.928395,0.789877,0.745574,0.789877
f1-score,0.669880,0.845894,0.789877,0.757887,0.779214
support,247.000000,405.000000,0.789877,652.000000,652.000000


In [52]:
gbt_pipeline = pipelines[3]

gbt_pipeline.fit(X_train, y_train)

y_pred = gbt_pipeline.predict(X_test)

gbt_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Gradient Boosted Trees Classification Report")
gbt_report

C:\Users\andre\AppData\Roaming\Python\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\andre\AppData\Roaming\Python\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Python312\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateP

Gradient Boosted Trees Classification Report


,Academic,Practitioner,accuracy,macro avg,weighted avg
precision,0.738197,0.821002,0.791411,0.779600,0.789633
recall,0.696356,0.849383,0.791411,0.772869,0.791411
f1-score,0.716667,0.834951,0.791411,0.775809,0.790141
support,247.000000,405.000000,0.791411,652.000000,652.000000


## Save Results

In [40]:
result_df.to_csv("../results/binary_baseline_model_results.csv", index=False)